In [1]:
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd 
import json
import random
from PIL import Image
import PIL.ImageOps    

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory

cudnn.benchmark = True
plt.ion()   

## Preprocessing

In [2]:
!mkdir dataset
!tar -xf ssbi-dataset-v1.0.0.zip -C dataset

A subdirectory or file dataset already exists.


In [2]:
train_json_path = "dataset/annotations/instances_train.json"
val_json_path = "dataset/annotations/instances_val.json"

In [3]:
#given a path to a coco json return a df containing filename, category_id, bounding box 
def coco_to_df(path):
    with open(path) as f:
        j = json.load(f)

    #put annotations and images into their own dataframs 
    annotations = pd.DataFrame(j['annotations'])
    images = pd.DataFrame(j['images'])

    #merge the two to get all important info and filter for only categories 6 or 7 forged or genuine 
    df1 = annotations[["image_id", "category_id", "bbox"]]
    df2 = images[["id", "file_name"]]
    df2 = df2.rename(columns={"id": "image_id"})
    final = df1.merge(df2, on='image_id')

    final = final[final['category_id'].isin([6, 7])].reset_index(drop=True)
    final['category_id'] = final['category_id'].replace(6, 1)
    final['category_id'] = final['category_id'].replace(7, 0)
    return final

In [4]:
train_df = coco_to_df(train_json_path)
val_df = coco_to_df(val_json_path)

display(train_df)
display(val_df)
display(len(train_df))
display(len(val_df))

,image_id,category_id,bbox,file_name
0,1,1,"[2574, 856, 284, 307]",check_009_001_F1_0.jpg
1,2,1,"[1671, 599, 166, 180]",check_008_001_F1_0.jpg
2,5,1,"[746, 262, 98, 106]",check_007_001_F1_0.jpg
3,7,1,"[655, 239, 92, 85]",check_004_001_F1_1.jpg
4,9,1,"[717, 262, 114, 106]",check_007_001_F1_1.jpg
...,...,...,...,...
3047,4352,0,"[818, 331, 109, 70]",check_012_019_G3_6.jpg
3048,4353,0,"[1036, 417, 149, 96]",check_016_019_G3_6.jpg
3049,4355,0,"[720, 242, 148, 95]",check_020_019_G3_6.jpg
3050,4357,0,"[872, 261, 93, 70]",check_017_019_G3_7.jpg


,image_id,category_id,bbox,file_name
0,3,1,"[1592, 410, 165, 179]",check_001_001_F1_0.jpg
1,4,1,"[1212, 493, 174, 188]",check_002_001_F1_0.jpg
2,6,1,"[1857, 553, 224, 207]",check_010_001_F1_1.jpg
3,8,1,"[933, 493, 203, 188]",check_002_001_F1_1.jpg
4,10,1,"[1141, 563, 194, 180]",check_005_001_F1_1.jpg
...,...,...,...,...
1303,4349,0,"[698, 294, 55, 39]",check_019_019_G3_5.jpg
1304,4353,0,"[547, 244, 95, 61]",check_018_019_G3_6.jpg
1305,4355,0,"[696, 242, 126, 95]",check_020_019_G3_7.jpg
1306,4358,0,"[747, 331, 93, 70]",check_012_019_G3_7.jpg


3052

1308

In [5]:
#takes in a dataframe of image and will crop and send back image with id (tensor, id)
class SSBI_Dataset(Dataset):
    def __init__(self, df, path, transform=None):
        self.df = df
        self.path = path 
        self.transform = transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        row = self.df.iloc[index]
        image = Image.open(os.path.join(self.path, row["file_name"])).convert("RGB")

        #crop based on bounding box
        x, y, w, h = row['bbox']
        image = image.crop([x, y, x+w, y+h])

        label = int(row["category_id"])
    
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [6]:
train_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

In [7]:
train_dataset = SSBI_Dataset(train_df, "dataset/train", train_transform)
val_dataset = SSBI_Dataset(val_df, "dataset/val", val_transform)

In [8]:
print(len(train_dataset))
print(len(val_dataset))

3052
1308


In [9]:
image, label = train_dataset[0]
print(image, label)

tensor([[[1.0844, 1.0502, 0.9646,  ..., 0.8789, 0.8789, 0.8789],
         [1.0844, 1.0502, 0.9646,  ..., 0.8789, 0.8789, 0.8789],
         [1.1015, 1.0673, 0.9817,  ..., 0.8789, 0.8789, 0.8789],
         ...,
         [0.9132, 0.9132, 0.8961,  ..., 0.6734, 0.6906, 0.7077],
         [0.9132, 0.9132, 0.8961,  ..., 0.8618, 0.8789, 0.8961],
         [0.9132, 0.9132, 0.8961,  ..., 0.9132, 0.9303, 0.9474]],

        [[1.1155, 1.1155, 1.1331,  ..., 1.1506, 1.1506, 1.1506],
         [1.1155, 1.1155, 1.1331,  ..., 1.1506, 1.1506, 1.1506],
         [1.1155, 1.1155, 1.1331,  ..., 1.1506, 1.1506, 1.1506],
         ...,
         [1.1856, 1.1856, 1.2031,  ..., 0.9755, 1.0105, 1.0105],
         [1.1856, 1.1856, 1.1856,  ..., 1.1331, 1.1681, 1.1856],
         [1.1856, 1.1856, 1.1856,  ..., 1.1856, 1.2206, 1.2381]],

        [[2.1694, 2.1868, 2.2391,  ..., 2.3088, 2.3088, 2.3088],
         [2.1694, 2.1868, 2.2391,  ..., 2.3088, 2.3088, 2.3088],
         [2.1868, 2.2043, 2.2566,  ..., 2.3088, 2.3088, 2.

## Train ResNet 

In [10]:
dataloaders = {'train' : torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=0), 
               'val' : torch.utils.data.DataLoader(val_dataset, batch_size=4, shuffle=True, num_workers=0)}

dataset_sizes = {'train' : len(train_dataset), 'val' : len(val_dataset)}

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device}")

Using cuda


In [11]:
#from pytorch documentation
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        best_acc = 0.0

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                if phase == 'train':
                    scheduler.step()

                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double() / dataset_sizes[phase]

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                # deep copy the model
                if phase == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    torch.save(model.state_dict(), best_model_params_path)

            print()

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val Acc: {best_acc:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path, weights_only=True))
    return model

In [12]:
#from pytorch documentation
def train_model(model, criterion, optimizer, scheduler, patience=7, num_epochs=25):
    since = time.time()
    best_val_loss = float('inf') 
    patience_counter = 0 

    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        best_acc = 0.0
        stop_training_flag = False

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                if phase == 'train':
                    scheduler.step()

                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double() / dataset_sizes[phase]

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                if phase == 'val':
                    
                    if epoch_acc > best_acc:
                        best_acc = epoch_acc
                        torch.save(model.state_dict(), best_model_params_path)
                        
                    if epoch_loss < best_val_loss:
                        best_val_loss = epoch_loss
                        patience_counter = 0 
                    else:
                        patience_counter += 1 

                    if patience_counter >= patience:
                        print('\n' + '='*20)
                        print(f'EARLY STOPPING: Validation loss did not improve for {patience} epochs.')
                        print(f'Stopping at epoch {epoch}.')
                        print('='*20)
                        stop_training_flag = True
                        break 

            print()
            if stop_training_flag:
                break

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val Acc: {best_acc:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path, weights_only=True))
    return model

In [13]:
model_conv = torchvision.models.resnet34(weights='IMAGENET1K_V1')
for param in model_conv.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 2)

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

In [ ]:
model_conv = train_model(model_conv, criterion, optimizer_conv, exp_lr_scheduler, num_epochs=50)
torch.save(model_conv.state_dict(), 'resnet_best_weights.pth')

Epoch 0/49
----------
train Loss: 0.7051 Acc: 0.7018
val Loss: 0.5786 Acc: 0.7630

Epoch 1/49
----------
train Loss: 0.7286 Acc: 0.7038
val Loss: 0.5680 Acc: 0.7424

Epoch 2/49
----------
train Loss: 0.6695 Acc: 0.7163
val Loss: 0.6040 Acc: 0.7676

Epoch 3/49
----------
train Loss: 0.6516 Acc: 0.7231
val Loss: 0.5318 Acc: 0.7584

Epoch 4/49
----------
train Loss: 0.6459 Acc: 0.7225
val Loss: 0.5590 Acc: 0.7538

Epoch 5/49
----------
train Loss: 0.6420 Acc: 0.7251
val Loss: 0.6820 Acc: 0.7699

Epoch 6/49
----------
train Loss: 0.6611 Acc: 0.7287
val Loss: 0.5457 Acc: 0.7309

Epoch 7/49
----------
train Loss: 0.5286 Acc: 0.7513
val Loss: 0.5216 Acc: 0.7813

Epoch 8/49
----------
train Loss: 0.5200 Acc: 0.7664
val Loss: 0.5103 Acc: 0.7546

Epoch 9/49
----------
train Loss: 0.5083 Acc: 0.7631
val Loss: 0.5488 Acc: 0.7775

Epoch 10/49
----------
train Loss: 0.5053 Acc: 0.7638
val Loss: 0.4986 Acc: 0.7806

Epoch 11/49
----------
train Loss: 0.5006 Acc: 0.7723
val Loss: 0.4747 Acc: 0.7829

Ep

In [18]:
best_weights_path = 'resnet_best_weights.pth'

model_conv = torchvision.models.resnet34(weights='IMAGENET1K_V1')

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 2)

loaded_model = torch.load(best_weights_path, map_location=device, weights_only=False)
model_conv.load_state_dict(loaded_model.state_dict(), strict=False)

for param in model_conv.parameters():
    param.requires_grad = False

for param in model_conv.layer4.parameters():
    param.requires_grad = True

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

#layer4 and last layer
optimizer_conv = optim.SGD(
    [
        {'params': model_conv.fc.parameters(), 'lr': 0.001},  
        
        {'params': model_conv.layer4.parameters(), 'lr': 0.0001} 
    ], 
    momentum=0.9
)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

In [19]:
model_conv = train_model(model_conv, criterion, optimizer_conv, exp_lr_scheduler, num_epochs=100)
torch.save(model_conv.state_dict(), 'resnet_best_weights_2.pth')

Epoch 0/99
----------
train Loss: 0.4997 Acc: 0.7720
val Loss: 0.4503 Acc: 0.8028

Epoch 1/99
----------
train Loss: 0.4869 Acc: 0.7792
val Loss: 0.4223 Acc: 0.8180

Epoch 2/99
----------
train Loss: 0.4581 Acc: 0.7939
val Loss: 0.3861 Acc: 0.8303

Epoch 3/99
----------
train Loss: 0.4503 Acc: 0.8031
val Loss: 0.3751 Acc: 0.8379

Epoch 4/99
----------
train Loss: 0.4426 Acc: 0.8044
val Loss: 0.3593 Acc: 0.8479

Epoch 5/99
----------
train Loss: 0.4405 Acc: 0.8021
val Loss: 0.3466 Acc: 0.8517

Epoch 6/99
----------
train Loss: 0.4285 Acc: 0.8087
val Loss: 0.3294 Acc: 0.8624

Epoch 7/99
----------
train Loss: 0.4040 Acc: 0.8244
val Loss: 0.3155 Acc: 0.8662

Epoch 8/99
----------
train Loss: 0.4125 Acc: 0.8142
val Loss: 0.3338 Acc: 0.8700

Epoch 9/99
----------
train Loss: 0.4103 Acc: 0.8237
val Loss: 0.3204 Acc: 0.8723

Epoch 10/99
----------
train Loss: 0.4126 Acc: 0.8221
val Loss: 0.3287 Acc: 0.8593

Epoch 11/99
----------
train Loss: 0.4034 Acc: 0.8254
val Loss: 0.3293 Acc: 0.8677

Ep

In [20]:
best_weights_path = 'resnet_best_weights_2.pth'

model_conv = torchvision.models.resnet34(weights='IMAGENET1K_V1')

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 2)

model_conv.load_state_dict(torch.load(best_weights_path, map_location=device, weights_only=False), strict=False)

for param in model_conv.parameters():
    param.requires_grad = False

for param in model_conv.layer4.parameters():
    param.requires_grad = True

for param in model_conv.layer3.parameters():
    param.requires_grad = True

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

#layer4 and last layer
optimizer_conv = optim.SGD(
    [
        {'params': model_conv.fc.parameters(), 'lr': 0.001},  
        
        {'params': model_conv.layer4.parameters(), 'lr': 0.0001},

        {'params': model_conv.layer3.parameters(), 'lr': 0.00001}
    ], 
    momentum=0.9
)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

In [21]:
model_conv = train_model(model_conv, criterion, optimizer_conv, exp_lr_scheduler, num_epochs=100)
torch.save(model_conv.state_dict(), 'resnet_best_weights_3.pth')

Epoch 0/99
----------
train Loss: 0.4113 Acc: 0.8234
val Loss: 0.2943 Acc: 0.8777

Epoch 1/99
----------
train Loss: 0.3890 Acc: 0.8290
val Loss: 0.2826 Acc: 0.8853

Epoch 2/99
----------
train Loss: 0.3927 Acc: 0.8303
val Loss: 0.2725 Acc: 0.8907

Epoch 3/99
----------
train Loss: 0.3978 Acc: 0.8290
val Loss: 0.2745 Acc: 0.8937

Epoch 4/99
----------
train Loss: 0.3795 Acc: 0.8421
val Loss: 0.2528 Acc: 0.9037

Epoch 5/99
----------
train Loss: 0.3746 Acc: 0.8375
val Loss: 0.2595 Acc: 0.9014

Epoch 6/99
----------
train Loss: 0.3663 Acc: 0.8463
val Loss: 0.2349 Acc: 0.9098

Epoch 7/99
----------
train Loss: 0.3497 Acc: 0.8624
val Loss: 0.2295 Acc: 0.9121

Epoch 8/99
----------
train Loss: 0.3565 Acc: 0.8545
val Loss: 0.2419 Acc: 0.9044

Epoch 9/99
----------
train Loss: 0.3576 Acc: 0.8503
val Loss: 0.2494 Acc: 0.9052

Epoch 10/99
----------
train Loss: 0.3579 Acc: 0.8483
val Loss: 0.2335 Acc: 0.9136

Epoch 11/99
----------
train Loss: 0.3520 Acc: 0.8496
val Loss: 0.2419 Acc: 0.9067

Ep

In [22]:
best_weights_path = 'resnet_best_weights_3.pth'

model_conv = torchvision.models.resnet34(weights='IMAGENET1K_V1')

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 2)

model_conv.load_state_dict(torch.load(best_weights_path, map_location=device, weights_only=False), strict=False)

model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()

optimizer_conv = optim.SGD(model_conv.parameters(), lr=0.001, momentum=0.9)

In [23]:
model_conv = train_model(model_conv, criterion, optimizer_conv, exp_lr_scheduler, num_epochs=150)
torch.save(model_conv.state_dict(), 'resnet_best_weights_4.pth')

Epoch 0/149
----------
train Loss: 0.6418 Acc: 0.7382
val Loss: 0.7133 Acc: 0.6919

Epoch 1/149
----------
train Loss: 0.5502 Acc: 0.7782
val Loss: 0.3044 Acc: 0.8670

Epoch 2/149
----------
train Loss: 0.4963 Acc: 0.8077
val Loss: 0.2006 Acc: 0.9067

Epoch 3/149
----------
train Loss: 0.3995 Acc: 0.8365
val Loss: 0.2733 Acc: 0.9434

Epoch 4/149
----------
train Loss: 0.3809 Acc: 0.8453
val Loss: 0.1214 Acc: 0.9526

Epoch 5/149
----------
train Loss: 0.3345 Acc: 0.8699
val Loss: 0.0663 Acc: 0.9763

Epoch 6/149
----------
train Loss: 0.3106 Acc: 0.8784
val Loss: 0.0534 Acc: 0.9763

Epoch 7/149
----------
train Loss: 0.2962 Acc: 0.8788
val Loss: 0.0590 Acc: 0.9855

Epoch 8/149
----------
train Loss: 0.2945 Acc: 0.8896
val Loss: 0.0418 Acc: 0.9893

Epoch 9/149
----------
train Loss: 0.2821 Acc: 0.8988
val Loss: 0.0497 Acc: 0.9847

Epoch 10/149
----------
train Loss: 0.2279 Acc: 0.9220
val Loss: 0.0342 Acc: 0.9901

Epoch 11/149
----------
train Loss: 0.2263 Acc: 0.9119
val Loss: 0.0714 Acc

the model is as good as it can be